### laptime & rank &timediff dataset - fulltest

base: 13./laptime_rank_timediff_dataset-oracle-fulltest

+ add support for historical data, indy500 for different years
+ add track and lap status as target, preparing for StatusModel with deepAR

this notebook focuses on create the training set, while laptime2rank-evaluate will have testset codes specified for rank calculation

+ add new features, caution_laps_instint,lap_instint

Build a time series dataset across all the oval races, including laptime, rank. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

Change to a new dataset format that following telemetry dataset.

raw:
+  [(eventid, carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[laptime, rank, track_status, lap_status]], in shape of #totallaps x #featureCnt (padded by nan)

gluonts:

+ Rank2LaptimeDifference

rank prediction by elapsed time difference in a race

Since use laptime2rank is not very successful, and predicting rank directly is not as well, to combine the real value regression into rank prediction task might be a solution.

And, differences of the elapsed time among the cars takes correlations into input, that brings more chances to success.

time_behind_leader is accurate when the car runs in the same lap as the leader does, otherwise, the time is truncted. To get accurate differences, using elapsed time is needed.


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Load Data

In [2]:
import os
os.getcwd()

'/scratch_hdd/hpda/indycar/notebook/14.StatusModelII'

In [3]:
# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    #uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
    #                  "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
    #                  "last_pitted_lap","start_position","laps_led"], axis=1)
    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", 
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
        
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    #df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    #df = uni_ds[['car_number','completed_laps','rank',
    #             'rank_diff','time_diff',"current_status", "track_status", "lap_status",'elapsed_time']]
    
    df = uni_ds[['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']]
    
    return df

def make_lapstatus_data(dataset):
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)
    
    #pick up one of them
    onecar = dataset[dataset['car_number']==completed_car_numbers[0]]
    onecar = onecar.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    return onecar[['completed_laps','track_status']]
    

In [4]:
def load_data(event, year=0):
    #inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    if year>0:
        inputfile = '../data/final/C_'+ event +'-' + year + '.csv'
    else:
        inputfile = '../data/final/C_'+ event +'.csv'
    
    #outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    flagdata = make_lapstatus_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata, flagdata

### overall view of laptime scatter plots



In [5]:
def get_cardata(curcarno, ycol='time_diff'):
    car = acldata[acldata['car_number']==curcarno]
    #print(car['time_diff'].describe())
    
    cols=['completed_laps','rank','car_number','lap_status','track_status',
    'pit_stop_count','current_status','start_position']
    colid={key:idx for idx, key in enumerate(cols)}

    cardata = rankdata[rankdata['car_number'] == curcarno]

    carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
        ['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']].values]
    
    x = car['completed_laps'][1:].values
    y = car[ycol][1:].values

    pits=[]
    yellowflags=[]
    lastflag = 'x'
    for row in carstatus:
        lap = int(row[colid['completed_laps']])

        if row[colid['lap_status']]=='P':
            pits.append(lap)

        if row[colid['track_status']]=='Y':
            if lastflag != 'Y':       
                #start
                yellowflags.append(lap)
        else:
            if lastflag == 'Y':       
                #end
                yellowflags.append(lap)        
        lastflag = row[colid['track_status']]

    #pit lap
    pits = np.array(pits)
    #start, end lap
    #
    yellowflags = np.array(yellowflags)
    if (yellowflags.shape[0] % 2)==1:
        print('crash?:carno=', curcarno)
        yellowflags = []
    else:
        yellowflags = np.array(yellowflags).reshape((-1,2))    
    
    return car, x, y, pits, yellowflags

### bulid the dataset

In [6]:
def get_lapdata(acldata):
    """
    input:
        acldata['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']
    
        timediff: [car_number, completed_laps] -> elapsed time diff to leader
    output:
        lapdata = acldata[['car_number','completed_laps',
                           'time_diff','rank','track_status', 'lap_status','time_behind']].to_numpy()
    """
    COL_COMPLETED_LAPS = 1
    COL_ELAPSED_TIME = 6
    
    maxlap = np.max(acldata['completed_laps'].values)
    #'car_number','completed_laps','time_diff','rank','track_status', 'lap_status','time_behind'
    time_behind = []
    
    for lap in range(1, maxlap+1):
        this_lap = acldata[acldata['completed_laps']==lap][
            ['car_number','completed_laps','time_diff','rank',
             'track_status', 'lap_status','elapsed_time']].values
        
        min_elapsed_time = np.nanmin(this_lap[:,COL_ELAPSED_TIME].astype(np.float))
        #print(f'lap:{lap}, min_elapsed_time:{min_elapsed_time}')
        
        for row in this_lap:
            car_number = int(row[0])
            time_diff = row[2]
            rank = row[3]
            track_status = row[4]
            lap_status = row[5]
            
            timebehind = float(row[COL_ELAPSED_TIME]) - min_elapsed_time
            #
            time_behind.append([car_number, lap, time_diff,rank,track_status, lap_status,
                                timebehind, float(row[COL_ELAPSED_TIME])])
    
    #return
    lapdata = np.array(time_behind)
    return lapdata

def get_laptime_dataset(stagedata):
    """
    #add caution_laps_instint, laps_instint
    
    input: (alldata, rankdata, acldata, flagdata)
    output: laptime & rank data
    
    [(
    eventid,
    carids : rowid -> carno,
    datalist: #car_number x features x #totallaps (padded by Nan)
        entry: [[laptime, rank, track_status, lap_status,
                caution_laps_instint, laps_instint]]
    )]
    """
    laptime_data = []
    for event in stagedata.keys():
        
        print(f'start event: {event}')
        
        laptime_rec = []
        eventid = events_id[event]
        
        alldata, rankdata, acldata, flagdata = stagedata[event]
        carlist = set(acldata['car_number'])
        laplist = set(acldata['completed_laps'])
        totalcars = len(carlist)
        totallaps = len(laplist)
        


        #carnumber -> carid
        carids={key:idx for idx, key in enumerate(carlist)}
        decode_carids={idx:key for idx, key in enumerate(carlist)}

        #init
        lap_instint = {carids[x]:0 for x in carlist}
        caution_instint = {carids[x]:0 for x in carlist}        
        
        #array: car_number x lap
        #laptime = np.zeros((totalcars, totallaps-1))
        #rank = np.zeros((totalcars, totallaps-1))
        laptime = np.empty((totalcars, totallaps-1))
        rank = np.empty((totalcars, totallaps-1))
        laptime[:] = np.NaN
        rank[:] = np.NaN
        
        # features: laptime, rank, track_status, lap_status, timediff
        LAPTIME = 0
        RANK = 1
        TRACK_STATUS = 2
        LAP_STATUS = 3
        TIME_BEHIND = 4
        CAUTION_LAPS_INSTINT = 5, 
        LAPS_INSTINT = 6,
        ELAPSED_TIME = 7,
        
        featureCnt = 8
        datalist = np.empty((totalcars, featureCnt, totallaps-1))
        datalist[:] = np.NaN
        
        #lapdata = acldata[['car_number','completed_laps',
        #                   'time_diff','rank','track_status', 'lap_status','elapsed_time']].to_numpy()
        
        #'car_number','completed_laps','time_diff','rank','track_status', 'lap_status','time_behind'
        lapdata = get_lapdata(acldata)
        
        
        for row in lapdata:
            #completed_laps
            if int(row[1]) == 0:
                continue
                
            #add to data array
            car_number = carids[int(row[0])]
            completed_laps = int(row[1])-1
            time_diff = float(row[2])
            rank = int(row[3])
            track_status = 1 if row[4]=='Y' else 0
            lap_status = 1 if row[5]=='P' else 0
            time_behind = float(row[6])
            
            datalist[car_number, LAPTIME, completed_laps] = time_diff
            datalist[car_number, RANK, completed_laps] = rank
            datalist[car_number, TRACK_STATUS, completed_laps] = track_status
            datalist[car_number, LAP_STATUS, completed_laps] = lap_status
            datalist[car_number, TIME_BEHIND, completed_laps] = time_behind

            datalist[car_number, ELAPSED_TIME, completed_laps] = float(row[7])

            #stint status
            
            if track_status == 1:
                caution_instint[car_number] += 1
            lap_instint[car_number] += 1
            if lap_status == 1:
                #new stint
                lap_instint[car_number] = 0
                caution_instint[car_number] = 0
            
            datalist[car_number, LAPS_INSTINT, completed_laps] = lap_instint[car_number]
            datalist[car_number, CAUTION_LAPS_INSTINT, completed_laps] = caution_instint[car_number]
                
            
        #add one record
        laptime_data.append([eventid, decode_carids, datalist])
        # push this event into stage dataframe
        print('event=%s, records=%s'%(event, datalist.shape))
        
    
    return laptime_data

### load data

In [7]:
#
# parameters
#
#year = '2017'
#years = [2008 2009 2011 2012 2013 2014 2015 2016 2017 2018 2019]
year = '2018'
#event = 'Toronto'
#https://www.racing-reference.info/season-stats/2018/O/#
events_totalmiles=[256,500,372,268,500,310]
events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']

events_id={key:idx for idx, key in enumerate(events)}
#events = ['Indy500']
#events = ['Phoenix']

In [56]:
stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0
#years = ['2011','2012','2013', '2014', '2015', '2016', '2017']
years = ['2012','2013', '2014', '2015', '2016', '2017']
years = ['2012','2013','2014','2015','2016','2017']
years = ['2011']
years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
events = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(events)}
dbid = f'Indy500_{years[0]}_{years[-1]}'

for event in events:
    #dataid = f'{event}-{year}'
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event)

    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1
    


count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 5, 8, 9, 11, 12, 77, 14, 78, 18, 19, 20, 83, 22, 55, 25, 26}
#cars= 19
count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 25, 26, 27, 41, 55, 60, 63, 77, 78, 81, 83, 91, 98}
#cars= 33
Indy500-2013: carno=33, lapnum=201
count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {33, 2, 3, 34, 5, 68, 98, 8, 11, 12, 77, 14, 16, 17, 18, 21, 22, 25, 26, 28}
#cars= 20


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 33, 34, 41, 63, 67, 68, 77, 83, 91, 98}
#cars= 33
Indy500-2014: carno=33, lapnum=201
count of completed cars: 20
completed cars: [ 2  1 83  9 15 27  3  6 21 22 11  5 14 24 28 98 48  7 29 26]
cars: {1, 2, 3, 98, 5, 6, 7, 9, 11, 14, 15, 48, 83, 21, 22, 24, 26, 27, 28, 29}
#cars= 20
count of completed cars: 20
completed cars: [ 2  1 83  9 15 27  3  6 21 22 11  5 14 24 28 98 48  7 29 26]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 32, 41, 43, 48, 63, 83, 88, 98}
#cars= 33
Indy500-2015: carno=33, lapnum=201
count of completed cars: 17
completed cars: [98 26 21 10 42  6  5  9 11 12  3 77 27 15  8 41 35]
cars: {98, 3, 35, 5, 6, 8, 9, 10, 11, 12, 41, 42, 15, 77, 21, 26, 27}
#cars= 17
count of completed cars: 17
completed cars: [98 26 21 10 42  6 

In [57]:
laptime_data = get_laptime_dataset(stagedata)

import pickle
#stintdf.to_csv('laptime-%s.csv'%year)
#with open(f'laptime_rank_timediff_fulltest-oracle-{year}.pickle', 'wb') as f:
with open(f'laptime_rank_timediff_pit-oracle-{dbid}.pickle', 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = [global_carids, laptime_data]
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

start event: Indy500-2013
event=Indy500-2013, records=(33, 8, 200)
start event: Indy500-2014
event=Indy500-2014, records=(33, 8, 200)
start event: Indy500-2015
event=Indy500-2015, records=(33, 8, 200)
start event: Indy500-2016
event=Indy500-2016, records=(33, 8, 200)
start event: Indy500-2017
event=Indy500-2017, records=(33, 8, 200)
start event: Indy500-2018
event=Indy500-2018, records=(33, 8, 200)
start event: Indy500-2019
event=Indy500-2019, records=(33, 8, 200)


### convert to gluonts dataset

In [10]:
# start from here
import pickle
#with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
with open(f'laptime_rank_timediff_pit-oracle-{dbid}.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 

In [18]:

def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
COL_CAUTION_LAPS_INSTINT=5
COL_LAPS_INSTINT= 6
COL_ELAPSED_TIME= 7


MODE_ORACLE = 0
MODE_NOLAP = 1
MODE_NOTRACK = 2
MODE_TESTZERO = 4
MODE_TESTCURTRACK = 8
#MODE_STR={MODE_ORACLE:'oracle', MODE_NOLAP:'nolap',MODE_NOTRACK:'notrack',MODE_TEST:'test'}

def make_dataset(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       train_ratio = 0.8,
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        #rerank due to short ts removed
        if run_ts == COL_RANK:
            sel_rows = []
            for rowid in range(_data[2].shape[0]):
                # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
                rec = _data[2][rowid].copy()

                totallen = rec.shape[1]
                if ( totallen < train_len + prediction_length):
                    print(f'rerank a short ts: carid={_data[1][rowid]}，len={totallen}')
                    continue 
                else:
                    sel_rows.append(rowid)
                    
            #get selected matrix
            sel_idx = np.array(sel_rows)
            selmat = _data[2][sel_idx]
            
            mask = np.isnan(selmat[:,COL_RANK,:])
            
            idx = np.argsort(selmat[:,COL_RANK,:], axis=0)
            true_rank = np.argsort(idx, axis=0).astype(np.float)
            true_rank[mask] = np.nan
            
            #set it back
            _data[2][sel_idx][:,COL_RANK,:] = true_rank
            
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0
                
            # split and add to dataset record
            _train.append({'target': rec[run_ts,:train_len].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [rec[COL_TRACKSTATUS,:train_len],
                                   rec[COL_LAPSTATUS,:train_len]]
                          }
                          )
            
            # multiple test ts(rolling window as half of the prediction_length)
            test_rec_cnt = 0
            for endpos in range(totallen, train_len+prediction_length, 
                                -int(prediction_length/2)):
                
                track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                
                # test mode
                if test_flag(oracle_mode, MODE_TESTCURTRACK):
                    # since nan does not work, use cur-val instead
                    track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                    #track_rec[-prediction_length:] = random.randint(0,1)
                    #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                    lap_rec[-prediction_length:] = 0
                elif test_flag(oracle_mode, MODE_TESTZERO):
                    #set prediction part as nan
                    #track_rec[-prediction_length:] = np.nan
                    #lap_rec[-prediction_length:] = np.nan
                    track_rec[-prediction_length:] = 0
                    lap_rec[-prediction_length:] = 0                    
                
                _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [track_rec,lap_rec]
                            #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                            #       rec[COL_LAPSTATUS,:endpos]] 
                             }
                          )   
                test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def make_dataset_byevent(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       test_event = 'Indy500-2018',
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       half_moving_win = True,
                       train_ratio=0.8,
                       log_transform = False,
                       context_ratio = 0.,
                       dorerank = True
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        if events[_data[0]] == test_event:
            test_mode = True
        
        else:
            test_mode = False
            
            
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        
        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10
        
        print(f'====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        #rerank due to short ts removed
        if run_ts == COL_RANK and dorerank == True:
            sel_rows = []
            for rowid in range(_data[2].shape[0]):
                # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
                rec = _data[2][rowid].copy()
                #remove nan(only tails)
                nans, x= nan_helper(rec[run_ts,:])
                nan_count = np.sum(nans)             
                rec = rec[:, ~np.isnan(rec[run_ts,:])]
                
                totallen = rec.shape[1]
                if ( totallen < train_len + prediction_length):
                    print(f'rerank a short ts: carid={_data[1][rowid]}，len={totallen}')
                    continue 
                else:
                    sel_rows.append(rowid)
                    
            #get selected matrix
            sel_idx = np.array(sel_rows)
            selmat = _data[2][sel_idx]
            
            mask = np.isnan(selmat[:,COL_RANK,:])
            
            idx = np.argsort(selmat[:,COL_RANK,:], axis=0)
            true_rank = np.argsort(idx, axis=0).astype(np.float)
            true_rank[mask] = np.nan
            
            #set it back
            #if _data[0]==0:
            #    print('raw:')
            #    print(_data[2][:,COL_RANK,0])
            #    print('true_rank:')
            #    print(true_rank[:,0])
            #_data[2][sel_idx][:,COL_RANK,:] = true_rank       
            _data[2][sel_idx,COL_RANK,:] = true_rank       
            #if _data[0]==0:
            #    _view = _data[2][sel_idx]
            #    _view[:,COL_RANK,:] = true_rank
            #    print('view:')
            #    print(_data[2][:,COL_RANK,0])
            #    print(_view[:,COL_RANK,0])
            #    print('rerank:')
            #    print(_data[2][sel_idx][:,COL_RANK,0])
        
        
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            #first, get target a copy    
            # target can be COL_XXSTATUS
            target_val = rec[run_ts,:].copy().astype(np.float32)
            if log_transform:
                target_val = np.log(target_val + 1.0)
            
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0

            test_rec_cnt = 0
            if not test_mode:
                # all go to train set
                _train.append({'target': target_val, 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [rec[COL_TRACKSTATUS,:],
                                       rec[COL_LAPSTATUS,:]]
                              }
                              )
            else:
                # reset train_len
                if context_ratio != 0.:
                    # all go to train set
                    #add [0, context_len] to train set 
                    _train.append({'target': target_val[:context_len], 
                                    'start': start, 
                                    'feat_static_cat': static_cat,
                                    'feat_dynamic_real': [rec[COL_TRACKSTATUS,:context_len],
                                           rec[COL_LAPSTATUS,:context_len]]
                                  }
                                  )                    
                
                # testset
                # multiple test ts(rolling window as half of the prediction_length)

                step = -int(prediction_length/2) if half_moving_win else -prediction_length
                for endpos in range(totallen, context_len+prediction_length, 
                                    step):

                    track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                    lap_rec = rec[COL_LAPSTATUS, :endpos].copy()

                    # test mode
                    if test_flag(oracle_mode, MODE_TESTCURTRACK):
                        # since nan does not work, use cur-val instead
                        track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                        #track_rec[-prediction_length:] = random.randint(0,1)
                        #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                        lap_rec[-prediction_length:] = 0
                    elif test_flag(oracle_mode, MODE_TESTZERO):
                        #set prediction part as nan
                        #track_rec[-prediction_length:] = np.nan
                        #lap_rec[-prediction_length:] = np.nan
                        track_rec[-prediction_length:] = 0
                        lap_rec[-prediction_length:] = 0                    

                        
                    #target_val = rec[run_ts,:endpos].astype(np.float32)
                    _test.append({'target': target_val[:endpos], 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [track_rec,lap_rec]
                                #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                                #       rec[COL_LAPSTATUS,:endpos]] 
                                 }
                              )   
                    test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### bulid dataset

In [12]:
def make_gluontsdb(run_ts, dbname, train_ratio=0.8):
    #run on indy dataset
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq,
                                         useeid=useeid, run_ts=run_ts,
                                        train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    if useeid:
        #special for classic predictor only
        return
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-zero-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq,
                                         oracle_mode=MODE_TESTCURTRACK,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-curtrack-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)    
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-all-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK+MODE_TESTZERO,
                                         useeid=useeid, run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-zero-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-all-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-zero-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTCURTRACK,
                                         useeid=useeid, run_ts=run_ts,train_ratio=train_ratio)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-curtrack-f{freq}-t{prediction_length}-r{train_ratio}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)    

In [13]:
def make_gluontsdb_byevent(run_ts, dbname, test_event='Indy500-2018', 
                           log_transform = False,context_ratio=0.,dorerank = True):
    #run on indy dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq,
                                         useeid=useeid, run_ts=run_ts,
                                        test_event=test_event,log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    if useeid:
        #special for classic predictor only
        return
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                         log_transform =log_transform,
                                         context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq,
                                         oracle_mode=MODE_TESTCURTRACK,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                        log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-curtrack-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)    
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                        log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK+MODE_TESTZERO,
                                         useeid=useeid, run_ts=run_ts,
                                        test_event=test_event,log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                        log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTZERO,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                        log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP+MODE_TESTCURTRACK,
                                         useeid=useeid, run_ts=run_ts,
                                        test_event=test_event,log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-curtrack-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds) 

### create dbs

In [19]:
### split by event -- laptime

freq = "1min"
context_ratio=0.
#context_ratio=0.2
#context_ratio=0.4

_run_ts = COL_LAPTIME
_task_id = 'laptime'

_run_ts = COL_TIMEDIFF
_task_id = 'timediff'

_run_ts = COL_LAPSTATUS
_task_id = 'lapstatus'


useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb_byevent(_run_ts,_task_id,context_ratio=context_ratio)
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb_byevent(_run_ts,_task_id,context_ratio=context_ratio)

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb_byevent(_run_ts,_task_id,context_ratio=context_ratio)

====event:Indy500-2013, prediction_len=2,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:2

carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, nancount:0, test_reccnt:0
carno:25, totallen:200, nancount:0, test_reccnt:0
carno:26, totallen:200, nancount:0, test_reccnt:0
carno:27, totallen:199, nancount:1, test_reccnt:0
carno:41, totallen:198, nancount:2, test_reccnt:0
carno:55, totallen:200, nancount:0, test_reccnt:0
carno:60, totallen:192, nancount:8, test_reccnt:0
a short ts: carid=63，len=46
carno:77, totallen:200, nancount:0, test_reccnt:0
carno:78, totallen:200, nancount:0, test_reccnt:0
carno:81, totallen:193, nancount:7, test_reccnt:0
carno:83, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=91，len=44
carno:98, totallen:196, nancount:4, test_reccnt:0
====event:Indy500-2014, prediction_len=2,train_len=160, max_len=200, min_len=200,context_len=10
carno:2, totallen:200, nancount:0, test_reccnt:0

====event:Indy500-2013, prediction_len=2,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:2

====event:Indy500-2013, prediction_len=2,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:2

a short ts: carid=83，len=149
a short ts: carid=91，len=87
carno:98, totallen:200, nancount:0, test_reccnt:0
====event:Indy500-2015, prediction_len=2,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
carno:4, totallen:176, nancount:24, test_reccnt:0
carno:5, totallen:200, nancount:0, test_reccnt:0
carno:6, totallen:200, nancount:0, test_reccnt:0
carno:7, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=8，len=0
carno:9, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=10，len=151
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:200, nancount:0, test_reccnt:0
carno:17, totallen:175, nancount:25, test_reccnt:0
a short ts: carid=18，len=116
a short ts: carid=19，len=116
a short ts: carid=20，len=112
carno:21, totallen:200, nancount:0, test_reccnt:0
carno:22, t

...

carno:1, totallen:200, nancount:0, test_reccnt:14
a short ts: carid=3，len=146
carno:4, totallen:200, nancount:0, test_reccnt:14
carno:6, totallen:200, nancount:0, test_reccnt:14
carno:7, totallen:193, nancount:7, test_reccnt:14
carno:9, totallen:200, nancount:0, test_reccnt:14
a short ts: carid=10，len=57
carno:12, totallen:200, nancount:0, test_reccnt:14
a short ts: carid=13，len=67
carno:14, totallen:187, nancount:13, test_reccnt:13
carno:15, totallen:200, nancount:0, test_reccnt:14
carno:17, totallen:199, nancount:1, test_reccnt:14
a short ts: carid=18，len=137
carno:19, totallen:199, nancount:1, test_reccnt:14
carno:20, totallen:200, nancount:0, test_reccnt:14
carno:21, totallen:199, nancount:1, test_reccnt:14
carno:22, totallen:200, nancount:0, test_reccnt:14
carno:23, totallen:200, nancount:0, test_reccnt:14
a short ts: carid=24，len=154
carno:25, totallen:200, nancount:0, test_reccnt:14
carno:26, totallen:198, nancount:2, test_reccnt:14
carno:27, totallen:200, nancount:0, test_reccn

carno:28, totallen:200, nancount:0, test_reccnt:0
carno:29, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=32，len=112
a short ts: carid=41，len=175
a short ts: carid=43，len=0
carno:48, totallen:200, nancount:0, test_reccnt:0
carno:63, totallen:197, nancount:3, test_reccnt:0
carno:83, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=88，len=62
carno:98, totallen:200, nancount:0, test_reccnt:0
====event:Indy500-2016, prediction_len=30,train_len=160, max_len=200, min_len=200,context_len=60
a short ts: carid=2，len=63
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=100
carno:5, totallen:200, nancount:0, test_reccnt:0
carno:6, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=7，len=126
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:200, nancount:0, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
a short ts: 

a short ts: carid=41，len=175
a short ts: carid=43，len=0
carno:48, totallen:200, nancount:0, test_reccnt:0
carno:63, totallen:197, nancount:3, test_reccnt:0
carno:83, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=88，len=62
carno:98, totallen:200, nancount:0, test_reccnt:0
====event:Indy500-2016, prediction_len=30,train_len=160, max_len=200, min_len=200,context_len=60
a short ts: carid=2，len=63
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=100
carno:5, totallen:200, nancount:0, test_reccnt:0
carno:6, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=7，len=126
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:200, nancount:0, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=14，len=163
carno:15, totallen:200, nancount:0, test_reccnt:0
carno:16, totallen:195, nancount:5, test_reccnt:0
a short ts: 

====event:Indy500-2013, prediction_len=5,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:2

====event:Indy500-2013, prediction_len=20,train_len=160, max_len=200, min_len=200,context_len=40
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
a short ts: carid=7，len=178
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, nanc

In [15]:
### split by event -- rank

freq = "1min"
context_ratio=0.
#context_ratio=0.2
#context_ratio=0.4

dorerank = False

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb_byevent(COL_RANK,'rank',context_ratio=context_ratio,dorerank =dorerank)

    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb_byevent(COL_RANK,'rank',context_ratio=context_ratio,dorerank =dorerank)

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb_byevent(COL_RANK,'rank',context_ratio=context_ratio,dorerank =dorerank)

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, 

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, 

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, 

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, 

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, 

...

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=40
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
a short ts: carid=7，len=178
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, nancount:0, test_reccn

carno:27, totallen:200, nancount:0, test_reccnt:0
carno:28, totallen:200, nancount:0, test_reccnt:0
carno:29, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=32，len=112
a short ts: carid=41，len=175
a short ts: carid=43，len=0
carno:48, totallen:200, nancount:0, test_reccnt:0
carno:63, totallen:197, nancount:3, test_reccnt:0
carno:83, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=88，len=62
carno:98, totallen:200, nancount:0, test_reccnt:0
====event:Indy500-2016, train_len=160, max_len=200, min_len=200,context_len=60
a short ts: carid=2，len=63
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=100
carno:5, totallen:200, nancount:0, test_reccnt:0
carno:6, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=7，len=126
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:200, nancount:0, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancou

a short ts: carid=2，len=63
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=100
carno:5, totallen:200, nancount:0, test_reccnt:0
carno:6, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=7，len=126
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:200, nancount:0, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=14，len=163
carno:15, totallen:200, nancount:0, test_reccnt:0
carno:16, totallen:195, nancount:5, test_reccnt:0
a short ts: carid=18，len=115
carno:19, totallen:199, nancount:1, test_reccnt:0
a short ts: carid=20，len=98
carno:21, totallen:200, nancount:0, test_reccnt:0
carno:22, totallen:199, nancount:1, test_reccnt:0
a short ts: carid=24，len=93
a short ts: carid=25，len=119
carno:26, totallen:200, nancount:0, test_reccnt:0
carno:27, totallen:200, nancount:0, test_reccnt:0
carno:28, totallen:198, nanco

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
carno:7, totallen:178, nancount:22, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, 

====event:Indy500-2013, train_len=160, max_len=200, min_len=200,context_len=60
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=3
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=34
a short ts: carid=7，len=178
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:197, nancount:3, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:193, nancount:7, test_reccnt:0
carno:16, totallen:199, nancount:1, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:191, nancount:9, test_reccnt:0
carno:22, totallen:200, nancount:0, test_reccn

In [16]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
### laptime dataset

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime')
    

In [ ]:
### rank dataset

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_RANK,'rank')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_RANK,'rank')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_RANK,'rank')

In [ ]:
### rank dataset

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff')

In [ ]:
# train_ratio=0.5

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff',train_ratio=0.5)
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff',train_ratio=0.5)

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_TIMEDIFF,'timediff',train_ratio=0.5)

In [ ]:
### split by event

freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff')
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff')

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff')

In [ ]:
## log trans
log_transform = True


freq = "1min"

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff',log_transform = True)
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff',log_transform = True)

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb_byevent(COL_TIMEDIFF,'timediff',log_transform = True)

In [ ]:
### split by ratio=0.5 -- laptime

freq = "1min"
tratio = 0.7
tratio = 0.8

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

plens=[2,5,10,20,30]
for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime',train_ratio=tratio)
    
#special
interpolate = True
useeid = False
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime',train_ratio=tratio)

useeid = True
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
if useeid:
    cardinality = [len(global_carids), len(laptime_data)]
else:
    cardinality = [len(global_carids)]

for prediction_length in plens:
    make_gluontsdb(COL_LAPTIME,'laptime',train_ratio=tratio)

### test

In [ ]:
def test_a(a, bitflag):
    return (a & bitflag) ==  bitflag

test_a(5, 4)


In [ ]:
mat = np.random.randint(5, size=(10,2, 5)).astype(np.float)
mat[2,1,0] = np.nan
_idx = np.array([0,1,2,8])
selmat = mat[_idx]
selmat[:,1,:]

In [ ]:
mask = np.isnan(selmat[:,1,:])
mask

In [ ]:
true_rank.shape

In [ ]:
idx = np.argsort(selmat[:,1,:], axis=0)
true_rank = np.argsort(idx, axis=0).astype(np.float)
true_rank[mask] = np.nan
true_rank

In [ ]:
selmat[:,1,:] = true_rank
selmat

In [ ]:
mat[_idx]

In [ ]:
mat[_idx] = selmat

In [ ]:
mat[_idx]

In [ ]:
freq, prediction_length, cardinality = 0,0,0

def load_dataset(inputfile):
    global freq, prediction_length, cardinality

    with open(inputfile, 'rb') as f:
        # have to specify it.
        freq, prediction_length, cardinality,train_ds, test_ds = pickle.load(f, encoding='latin1')
    
    print(f"number of cars: {cardinality}")
    
    return train_ds, test_ds

train_ds, test_ds = load_dataset('rank-oracle-noip-eid-all-all-f1min-t30-rIndy500-gluonts-indy-2018.pickle')

In [ ]:
ds_iter =  iter(test_ds)
rec = next(ds_iter)
rec

In [ ]:
def debug_get_ranklist(train_ds):
    ds_iter =  iter(train_ds)
    ranklist = []
    for idx in range(len(train_ds)):
        rec = next(ds_iter)

        if rec['feat_static_cat'][1] == 0:
            ranklist.append(rec['target'][0])

    return np.array(sorted(ranklist))

In [ ]:
len(ranklist)

In [ ]:
def get_testds(dorerank):
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, 2,freq,
                                        useeid=True, run_ts=COL_RANK,
                                        test_event='Indy500',log_transform =False,
                                        context_ratio=0,
                                                dorerank=dorerank)
    return train_ds, test_ds

In [ ]:
rank_train, rank_ds = get_testds(True)

In [ ]:
rawrank_train, rawrank_ds = get_testds(False)

In [ ]:
debug_get_ranklist(rank_train)

In [ ]:
debug_get_ranklist(rawrank_train)

In [ ]:
debug_get_ranklist(train_ds)

In [ ]:
 #alldata, rankdata, acldata, flagdata = stagedata['Indy500-2011']

In [ ]:
set(acldata.car_number.values)

In [ ]:
acldata

In [ ]:
acldata[acldata['completed_laps']==187]

In [ ]:
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==186].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==187].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==188].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==189].car_number.values))))


In [27]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2014']

In [30]:
kannan = rankdata[rankdata['car_number']==10]
kannan[(kannan['completed_laps'] > 60) & (kannan['completed_laps']<70)]

,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
5092,10,10,F,61,2577.9406,40.9674,T,40.4981,22,4.5262,...,0.9545,0,A,40.4981,Active,G,1,20,10,1
5172,6,10,F,62,2619.4950,41.5544,T,40.4981,22,4.1976,...,0.8224,0,6,40.4981,Active,G,1,20,10,1
5283,3,10,F,63,2660.4248,40.9298,T,40.4981,22,2.7521,...,0.9311,0,3,40.4981,Active,G,1,20,10,1
5399,2,10,F,64,2701.3298,40.9050,T,40.4981,22,1.6321,...,1.6321,0,2,40.4981,Active,G,1,20,10,1
7005,32,10,F,65,3499.1579,797.8281,P,40.4981,22,24.7495,...,39.8184,8,20,40.4981,Active,G,2,41,10,1
7154,32,10,F,66,3565.4691,66.3112,T,40.4981,22,9.2227,...,23.4681,9,20,40.4981,Active,G,2,41,10,1
7248,32,10,F,67,3610.2211,44.7520,T,40.4981,22,13.1535,...,27.6159,9,20,40.4981,Active,G,2,41,10,1
7343,32,10,F,68,3651.1963,40.9752,T,40.4981,22,13.2349,...,27.8207,9,20,40.4981,Active,G,2,41,10,1
7429,32,10,F,69,3692.4117,41.2154,T,40.4981,22,13.6112,...,28.2414,9,20,40.4981,Active,G,2,41,10,1


In [46]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2013']
rankdata[(rankdata['car_number']==16) & (rankdata['completed_laps']<10)]

,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
20,20,16,13,0,2.7766,79.3742,T,0.0000,0,2.7766,...,2.7766,0,14,0.0000,Active,G,0,0,14,0
107,25,16,13,1,47.9618,45.1852,T,45.1852,1,5.4494,...,0.0341,0,19,45.1852,Active,G,0,0,14,0
185,25,16,13,2,90.4377,42.4759,T,42.4759,2,6.8528,...,0.2774,0,19,42.4759,Active,G,0,0,14,0
258,24,16,13,3,132.1525,41.7148,T,41.7148,3,7.5099,...,0.3469,0,18,41.7148,Active,G,0,0,14,0
345,23,16,13,4,196.0176,63.8651,T,41.7148,3,14.1331,...,0.5255,0,17,41.7148,Active,Y,0,0,14,0
471,28,16,13,5,336.6918,140.6742,P,41.7148,3,26.4029,...,2.3969,0,1C,41.7148,Active,Y,1,5,14,0
561,28,16,13,6,463.6968,127.0050,T,41.7148,3,29.4791,...,0.5088,0,1C,41.7148,Active,Y,1,5,14,0
633,28,16,13,7,524.3235,60.6267,T,41.7148,3,7.1228,...,0.0094,0,1C,41.7148,Active,G,1,5,14,0
710,29,16,13,8,567.9438,43.6203,T,41.7148,3,8.7344,...,0.1710,0,1D,41.7148,Active,G,1,5,14,0


In [53]:
rankdata[(rankdata['car_number']==16) & (rankdata['completed_laps']>88)& (rankdata['completed_laps']<97)]

,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
7111,18,16,13,89,4579.4683,41.2333,T,40.0519,19,7.0275,...,0.5453,0,12,40.0519,Active,G,3,3A,14,5
7202,10,16,13,90,4620.7358,41.2675,T,40.0519,19,6.3345,...,1.4957,0,A,40.0519,Active,G,3,3A,14,5
7430,26,16,13,91,4711.4957,90.7599,P,40.0519,19,2.1256,...,8.1903,0,1A,40.0519,Active,G,4,5B,14,5
7504,26,16,13,92,4769.2300,57.7343,T,40.0519,19,59.8599,...,14.2249,0,1A,40.0519,Active,G,4,5B,14,5
7616,26,16,13,93,4810.5582,41.3282,T,40.0519,19,33.8846,...,15.0441,0,1A,40.0519,Active,G,4,5B,14,5
7684,26,16,13,94,4851.8727,41.3145,T,40.0519,19,34.4120,...,15.8051,0,1A,40.0519,Active,G,4,5B,14,5
7813,27,16,13,95,4905.1311,53.2584,P,40.0519,19,5.1703,...,2.3107,0,1B,40.0519,Active,G,5,5F,14,5
7902,27,16,13,96,4962.4055,57.2744,T,40.0519,19,20.9156,...,18.0551,0,1B,40.0519,Active,G,5,5F,14,5


In [47]:
car16 = acldata[acldata['car_number']==16]


In [48]:
for idx in range(len(car16)):
    if(car16.iloc[idx].lap_status == 'P'):
        print(car16.iloc[idx].completed_laps)

5
38
58
91
95
120
151
180


In [49]:
for idx in range(len(car16)):
    if(car16.iloc[idx].track_status == 'Y'):
        print(car16.iloc[idx].completed_laps)

4
5
6
35
36
37
38
39
40
41
42
57
58
59
60
192
193
194
195
197
198


In [55]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2014']
car16 = acldata[acldata['car_number']==14]
car16[car16['lap_status']=='P']

,car_number,completed_laps,time_diff,rank,track_status,lap_status,elapsed_time
MyIdx,,,,,,,
2503,14,30,67.0756,19,G,P,1276.1343
5353,14,63,67.7364,12,G,P,2691.9810
8038,14,93,67.6730,18,G,P,3974.9730
10620,14,122,64.6061,16,G,P,5199.1175
13426,14,153,94.8046,14,Y,P,6650.8045
14877,14,170,108.0939,16,Y,P,7617.1779
16647,14,192,155.0246,16,Y,P,9194.4255
17120,14,193,164.0781,20,Y,P,9358.5036
